# SimRiskPlot

Plotting Routines for SimRisk

Ronald L. Rivest

November 18, 2020

This notebook provides plotting routines for the SimRisk package,
based on `matplotlib`.  We emphasize keeping everything in 
"data coordinates" 
(see https://riptutorial.com/matplotlib/topic/4566/coordinates-systems).

In [1]:
# See https://github.com/JuliaPy/PyPlot.jl
#     https://matplotlib.org/api/pyplot_api.html
import PyPlot
const plt = PyPlot

using Formatting

In [2]:
### HORIZONTAL LAYOUT
nNamedCols = 0       # number of named people
nAnonymousCols = 0   # 1 if anonymous used, else 0
nCols = 0            # sum of previous two

leftMargin = 0.50    # data coordinates
# columns are 1-indexed:  1..nCols
colWidth = 0.80      # in data coordinates
titleX = 0.50        
leftColX(col) = leftMargin+colWidth*(col-1)
centerColX(col) = leftMargin+colWidth*(col-0.5)
rightColX(col) = leftMargin+colWidth*col

commentX = 0.81      # to be reassigned by computeSize

0.81

In [3]:
### VERTICAL LAYOUT
dataHeight = 10.00

titleHeight = 1.00
idHeight = 1.00
rowHeight = 1.00

titleY = dataHeight - rowHeight
idY = titleY - 1.20*rowHeight

# data rows are numbered 1...nRows
nRows = 0            # to be reassigned by computeSize
rowY(row) = idY - rowHeight*row

function computeSize(m::Model)
    global nNamedCols, nAnonymousCols, nCols, nRows, commentX
    nNamedCols = length(m.people)
    contactEvents = [e for e in m.events if e isa ContactEvent]
    nAnonymousPeople = [e.anonymousNumber for e in contactEvents]
    nAnonymousCols = min(1, sum([0; nAnonymousPeople]))
    nCols = nNamedCols+nAnonymousCols
    commentX = leftColX(nCols+1)
    nRows = length([e for e in m.events 
                      if !(e isa NewPersonEvent)])   
end
            
function initPlot()

    ymin = rowY(nRows+2)
    ymax = dataHeight
    yrange = ymax - ymin 
                                
    fig = plt.figure(figsize = [max(nCols+2,4),yrange*0.40])
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0])   
                                
    ax.set_ylim([ymin,ymax])

    xmin = 0
    xmax = rightColX(nCols+2)  
                                
    ax.set_xlim([xmin, xmax])

    ax.axis("off")    # turns off framebox
end

function titlePlot(m::Model)
    title = m.name
    ax = plt.gca()
    if title == ""
        title = "(Example)"
    end
    plt.text(titleX, titleY,
             title, 
             horizontalalignment="left",
             verticalalignment="center",
             size="x-large",
             transform=ax.transData)
end

function idsPlot(m::Model)
    ax = plt.gca()    # get current axes
    for (col, p) in enumerate(m.people)
        # 0,0 is lower-left, 1,1 is upper-right
        plt.text(centerColX(col), 
                 idY, 
                 p.personID,
                 horizontalalignment="center",
                 verticalalignment="center",
                 bbox=Dict(:facecolor=>"white", :alpha=>1.0, :linewidth=>0.0),
                 transform=ax.transData)
    end
    if nAnonymousCols>0
        plt.text(centerColX(nCols),
                 idY,
                 "?(anon)",
                 horizontalalignment="center",
                 verticalalignment="center",
                 bbox=Dict(:facecolor=>"white", :alpha=>1.0, :linewidth=>0.0),    
                 transform=ax.transData)               
    end
end    

function vlinesPlot(m::Model)
    ax = plt.gca()
    for col in 1:nNamedCols
        # 0 <= y <= 1; 0 is at bottom
        plt.vlines(x=centerColX(col), 
                   ymin = rowY(nRows),
                   ymax = idY)
    end
    if nAnonymousCols != 0
        plt.vlines(x=centerColX(nCols),
                   ymin = rowY(nRows),
                   ymax = idY, 
                   linestyle="dashed")
    end
end

function statsPlot(m::Model)
    # ax = plt.gca()
    t = 0
    row = 0                               
    for e in m.events
        !(e isa NewPersonEvent) && (row += 1)
        if e isa StatsEvent
            t += 1
            plt.hlines(y=rowY(row),
                       xmin=centerColX(1), 
                       xmax=rightColX(nCols)-0.10*colWidth,
                       color="red",
                       lw=0.4)
            for (col, p) in enumerate(m.people)
                infectedRuns = get(m.stats, (p, t, "infected runs"), 0)
                totalRuns = get(m.stats, (p, t, "total runs"), 0)
                percentRunsInfected = 100*infectedRuns/totalRuns
                percentRunsInfected = Int(round(percentRunsInfected))
                plt.text(centerColX(col), 
                         rowY(row),
                         fmt("2d", percentRunsInfected)*" %",
                         fontsize=6,
                         horizontalalignment="center",
                         verticalalignment="center",
                         bbox=Dict(:facecolor=>"white", :alpha=>1.0, :linewidth=>0.2))
            end
            totalRuns = get(m.stats, (m.people[1], t, "total runs"), 0)                                        
            plt.text(commentX,
                     rowY(row),
                     format("Stats: infection probability ({} runs)", Int64(totalRuns)),
                     fontsize=6,
                     horizontalalignment="left",
                     verticalalignment="center")
        end
    end
end
                            
function newDayPlot(m::Model)
    ax = plt.gca()
    t = 1
    row = 0                                
    for e in m.events
        !(e isa NewPersonEvent) && (row += 1)
        if e isa NewDayEvent
            plt.hlines(y=rowY(row),
                       xmin=leftColX(1), 
                       xmax=rightColX(nCols)-0.10*colWidth,
                       linestyle="dotted")
            t += 1
            plt.text(commentX,
                     rowY(row),
                     format("Begin day {}", t),
                     fontsize=6,
                     horizontalalignment="left",
                     verticalalignment="center")
        end
    end
end
            
function testPlot(m::Model)
    ax = plt.gca()
    t = 0
    row = 0                                   
    for e in m.events
        !(e isa NewPersonEvent) && (row += 1)
        if e isa TestEvent
            col = findfirst(p -> (p.personID == e.person.personID), m.people)
            result = e.reportedTestResult ? "infected" : "not infected"
            plt.text(commentX, 
                     rowY(row),
                     format("{} test result: {}", 
                            e.testType.name,
                            result),
                     fontsize=6,
                     horizontalalignment="left",
                     verticalalignment="center")
            plt.text(centerColX(col),
                     rowY(row),
                     "Test",
                     fontsize=6,
                     horizontalalignment="center",
                     verticalalignment="center",
                     bbox=Dict(:facecolor=>"white", :alpha=>1.0))
        end
        t += 1
    end
end
            
function symptomOnsetPlot(m::Model)
    ax = plt.gca()
    t = 0
    row = 0                                  
    for e in m.events
        !(e isa NewPersonEvent) && (row += 1)
        if e isa SymptomOnsetEvent
            col = findfirst(p -> (p.personID == e.person.personID), m.people)
            plt.text(commentX, 
                     rowY(row),
                     format("SymptomOnset: {}", 
                            e.reportedSymptomOnsetResult),
                     fontsize=6,
                     horizontalalignment="left",
                     verticalalignment="center")
            plt.text(centerColX(col),
                     rowY(row),
                     "SO",
                     fontsize=6,
                     horizontalalignment="center",
                     verticalalignment="center",
                     bbox=Dict(:facecolor=>"white", :alpha=>1.0))
        end
        t += 1
    end
end 

function contactPlot(m::Model)
    row = 0                               
    for e in m.events
        !(e isa NewPersonEvent) && (row += 1)
        if e isa ContactEvent
            cols = findall(x->m.people[x] in e.people, 1:length(m.people))
            mincol = minimum(cols)
            maxcol = maximum(cols)
            if e.anonymousNumber > 0
                maxcol = nCols
            end
            plt.hlines(y=rowY(row),
                       xmin=centerColX(mincol), 
                       xmax=centerColX(maxcol),
                       color="black",
                       lw=1.75)
            for col in cols
                plt.text(centerColX(col), 
                         rowY(row),
                         "*",
                         fontsize=3,
                         horizontalalignment="center",
                         verticalalignment="center",
                         bbox=Dict(:facecolor=>"black", :alpha=>1.0, :linewidth=>1.0))
            end
            if e.anonymousNumber>0
                plt.text(centerColX(nCols), 
                         rowY(row),
                         format("x {}",e.anonymousNumber),
                         fontsize=6,
                         horizontalalignment="center",
                         verticalalignment="center",
                         bbox=Dict(:facecolor=>"white", :alpha=>1.0, :linewidth=>1.0))
            end
            plt.text(commentX,
                     rowY(row),
                     format("Contact {} minutes", e.minutes),
                     fontsize=6,
                     horizontalalignment="left",
                     verticalalignment="center")
        end
    end
end

function simPlot(m::Model; save="")
    computeSize(m)   
    initPlot()                
    titlePlot(m)
    vlinesPlot(m)
    idsPlot(m)
    newDayPlot(m)
    testPlot(m) 
    symptomOnsetPlot(m) 
    statsPlot(m)
    contactPlot(m)                                
    if save!=""
        plt.savefig(save * ".png")                            
    end
end

UndefVarError: UndefVarError: Model not defined